In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.model_selection import train_test_split
  

In [2]:
# Read csv file into DataFrame
from operator import index


df = pd.read_csv("Resources/Google-Playstore.csv")
df.head()

,App Name,App Id,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,...,Developer Website,Developer Email,Released,Last Updated,Content Rating,Privacy Policy,Ad Supported,In App Purchases,Editors Choice,Scraped Time
0,Gakondo,com.ishakwe.gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,...,https://beniyizibyose.tk/#/,jean21101999@gmail.com,"Feb 26, 2020","Feb 26, 2020",Everyone,https://beniyizibyose.tk/projects/,False,False,False,2021-06-15 20:19:35
1,Ampere Battery Info,com.webserveis.batteryinfo,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,...,https://webserveis.netlify.app/,webserveis@gmail.com,"May 21, 2020","May 06, 2021",Everyone,https://dev4phones.wordpress.com/licencia-de-uso/,True,False,False,2021-06-15 20:19:35
2,Vibook,com.doantiepvien.crm,Productivity,0.0,0.0,50+,50.0,58,True,0.0,...,NaN,vnacrewit@gmail.com,"Aug 9, 2019","Aug 19, 2019",Everyone,https://www.vietnamairlines.com/vn/en/terms-an...,False,False,False,2021-06-15 20:19:35
3,Smart City Trichy Public Service Vehicles 17UC...,cst.stJoseph.ug17ucs548,Communication,5.0,5.0,10+,10.0,19,True,0.0,...,http://www.climatesmarttech.com/,climatesmarttech2@gmail.com,"Sep 10, 2018","Oct 13, 2018",Everyone,NaN,True,False,False,2021-06-15 20:19:35
4,GROW.me,com.horodyski.grower,Tools,0.0,0.0,100+,100.0,478,True,0.0,...,http://www.horodyski.com.pl,rmilekhorodyski@gmail.com,"Feb 21, 2020","Nov 12, 2018",Everyone,http://www.horodyski.com.pl,False,False,False,2021-06-15 20:19:35


In [3]:
# Drop unnessasery columns
ml_df = df.drop(columns=["App Id", "Currency", "Minimum Android", "Developer Id", "Developer Website", "Developer Email", 
"Released", "Last Updated", "Privacy Policy", "Scraped Time"])
ml_df.head()

,App Name,Category,Rating,Rating Count,Installs,Minimum Installs,Maximum Installs,Free,Price,Size,Content Rating,Ad Supported,In App Purchases,Editors Choice
0,Gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,10M,Everyone,False,False,False
1,Ampere Battery Info,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,2.9M,Everyone,True,False,False
2,Vibook,Productivity,0.0,0.0,50+,50.0,58,True,0.0,3.7M,Everyone,False,False,False
3,Smart City Trichy Public Service Vehicles 17UC...,Communication,5.0,5.0,10+,10.0,19,True,0.0,1.8M,Everyone,True,False,False
4,GROW.me,Tools,0.0,0.0,100+,100.0,478,True,0.0,6.2M,Everyone,False,False,False


In [4]:
ml_df = ml_df[ml_df["Installs"].notna()]
ml_df = ml_df[ml_df["Size"].notna()]
ml_df = ml_df[ml_df["Rating"].notna()]
ml_df = ml_df[ml_df["Rating Count"].notna()]


In [5]:
ml_df.dtypes

App Name             object
Category             object
Rating              float64
Rating Count        float64
Installs             object
Minimum Installs    float64
Maximum Installs      int64
Free                   bool
Price               float64
Size                 object
Content Rating       object
Ad Supported           bool
In App Purchases       bool
Editors Choice         bool
dtype: object

In [6]:
ml_df = ml_df.drop_duplicates(subset=["App Name"])
print(F"Duplicates: {ml_df.duplicated().sum()}")

Duplicates: 0


In [7]:
ml_df = ml_df.applymap(lambda x: x.replace('\'', '') if (isinstance(x, str)) else x)
ml_df = ml_df.applymap(lambda x: x.replace('"', '') if (isinstance(x, str)) else x)
ml_df.dropna(axis=0, how='any', inplace=True)
ml_df.isnull().sum()

App Name            0
Category            0
Rating              0
Rating Count        0
Installs            0
Minimum Installs    0
Maximum Installs    0
Free                0
Price               0
Size                0
Content Rating      0
Ad Supported        0
In App Purchases    0
Editors Choice      0
dtype: int64

In [8]:
# Replace the spaces in the column names with underscores
ml_df.columns = [c.replace(' ', '_') for c in ml_df.columns]

In [9]:
def value_to_float(x):

    x = str(x).strip().replace(',', '').replace('Varies with device','0')

    if 'M' in str(x):
        x = x.replace('M', '') 

    if 'k' in str(x):
        x = x.replace('k', '')
        x = float(x) * 0.0009765625 

    if 'G' in str(x):
        x = str(x).replace('G', '')
        x = float(x) * 1024

    return float(x)

ml_df["Size"] = ml_df["Size"].apply(value_to_float)
ml_df.head()

,App_Name,Category,Rating,Rating_Count,Installs,Minimum_Installs,Maximum_Installs,Free,Price,Size,Content_Rating,Ad_Supported,In_App_Purchases,Editors_Choice
0,Gakondo,Adventure,0.0,0.0,10+,10.0,15,True,0.0,10.0,Everyone,False,False,False
1,Ampere Battery Info,Tools,4.4,64.0,"5,000+",5000.0,7662,True,0.0,2.9,Everyone,True,False,False
2,Vibook,Productivity,0.0,0.0,50+,50.0,58,True,0.0,3.7,Everyone,False,False,False
3,Smart City Trichy Public Service Vehicles 17UC...,Communication,5.0,5.0,10+,10.0,19,True,0.0,1.8,Everyone,True,False,False
4,GROW.me,Tools,0.0,0.0,100+,100.0,478,True,0.0,6.2,Everyone,False,False,False


In [10]:
ml_df["Size"].dtypes

dtype('float64')

In [11]:
def object_to_int(x):

    x = str(x).strip().replace('+', '')

    if '+' in str(x):
        x = x.replace('+', '') 
    if ',' in str(x):
        x = x.replace(',', '')
    
    return int(x)

ml_df["Installs"] = ml_df["Installs"].apply(object_to_int)
ml_df.head()



,App_Name,Category,Rating,Rating_Count,Installs,Minimum_Installs,Maximum_Installs,Free,Price,Size,Content_Rating,Ad_Supported,In_App_Purchases,Editors_Choice
0,Gakondo,Adventure,0.0,0.0,10,10.0,15,True,0.0,10.0,Everyone,False,False,False
1,Ampere Battery Info,Tools,4.4,64.0,5000,5000.0,7662,True,0.0,2.9,Everyone,True,False,False
2,Vibook,Productivity,0.0,0.0,50,50.0,58,True,0.0,3.7,Everyone,False,False,False
3,Smart City Trichy Public Service Vehicles 17UC...,Communication,5.0,5.0,10,10.0,19,True,0.0,1.8,Everyone,True,False,False
4,GROW.me,Tools,0.0,0.0,100,100.0,478,True,0.0,6.2,Everyone,False,False,False


In [12]:
ml_df["Installs"].dtypes

dtype('int64')

In [13]:
# Print out the Category value counts
category_counts = ml_df.Category.value_counts()
category_counts

Education                  226779
Music & Audio              145328
Business                   138881
Tools                      130057
Entertainment              127920
Lifestyle                  111326
Books & Reference          107393
Health & Fitness            78982
Personalization             76817
Productivity                74053
Shopping                    73067
Food & Drink                70718
Travel & Local              65182
Finance                     62229
Arcade                      48248
Communication               45747
Sports                      45728
Puzzle                      45713
Casual                      45530
Social                      41842
News & Magazines            40858
Medical                     30895
Photography                 29833
Action                      25757
Maps & Navigation           25605
Simulation                  22100
Adventure                   21933
Educational                 20008
Auto & Vehicles             17438
Art & Design  

In [14]:
# Determine which values to replace
replace_categories = list(category_counts[category_counts < 25000].index)

# Replace in DataFrame
for category in replace_categories:
    ml_df.Category = ml_df.Category.replace(category,"Other")


# Check to make sure binning was successful
ml_df.Category.value_counts()

Other                242609
Education            226779
Music & Audio        145328
Business             138881
Tools                130057
Entertainment        127920
Lifestyle            111326
Books & Reference    107393
Health & Fitness      78982
Personalization       76817
Productivity          74053
Shopping              73067
Food & Drink          70718
Travel & Local        65182
Finance               62229
Arcade                48248
Communication         45747
Sports                45728
Puzzle                45713
Casual                45530
Social                41842
News & Magazines      40858
Medical               30895
Photography           29833
Action                25757
Maps & Navigation     25605
Name: Category, dtype: int64

In [15]:
ml_df = ml_df[ml_df['App_Name'].map(lambda x: x.isascii())]

In [17]:
google_app_info = ml_df.drop(columns=['Minimum_Installs', 'Installs', 'Maximum_Installs', 'Size', 'Ad_Supported', 'In_App_Purchases'])
google_app_info.to_csv("Resources/google_app_info.csv", index=False)
google_app_info.head()

,App_Name,Category,Rating,Rating_Count,Free,Price,Content_Rating,Editors_Choice
0,Gakondo,Other,0.0,0.0,True,0.0,Everyone,False
1,Ampere Battery Info,Tools,4.4,64.0,True,0.0,Everyone,False
2,Vibook,Productivity,0.0,0.0,True,0.0,Everyone,False
3,Smart City Trichy Public Service Vehicles 17UC...,Communication,5.0,5.0,True,0.0,Everyone,False
4,GROW.me,Tools,0.0,0.0,True,0.0,Everyone,False


In [ ]:
app_installs = ml_df.drop(columns=["Category", "Rating", "Rating_Count", "Free", "Price", "Content_Rating", "Editors_Choice"])
app_installs.to_csv("Resources/app_installs.csv", index=False)
app_installs.head()

In [ ]:
# Import dependencies to connect to database
import sqlalchemy 
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database
from local_settings import postgresql as settings

In [ ]:
# Create function for get engine
def get_engine(user, passwd, host, port, db):
    url = f"postgresql://{user}:{passwd}@{host}:{port}/{db}"
    engine = create_engine(url, pool_size=50, echo=False)
    return engine

In [ ]:
# Create engine
engine = get_engine(settings['pguser'],
                    settings['pgpasswd'],
                    settings['pghost'],
                    settings['pgport'],
                    settings['pgdb'])

In [ ]:
engine.url

In [ ]:
# Function to create engine
def get_engine_from_settings():
    keys = ['pguser', 'pgpasswd', 'pghost', 'pgport', 'pgdb']
    if not all(key in keys for key in settings.keys()):
        raise Exception('Bad config file')

    return get_engine(settings['pguser'],
                    settings['pgpasswd'],
                    settings['pghost'],
                    settings['pgport'],
                    settings['pgdb'])

In [ ]:
# Function to create session
def get_session():
    engine = get_engine_from_settings()
    session = sessionmaker(bind=engine)()
    return session

In [ ]:
# Create session
session = get_session()